# Flux at Low Order channel

We estimate the flux at the Low Order WFS as



In [1]:
import numpy as np
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from arte.photometry.morfeo_transmissive_systems import MorfeoLowOrderChannelTransmissiveSystem_002
from arte.photometry.eso_sky_calc import EsoSkyCalc
from arte.photometry.normalized_star_spectrum import get_normalized_star_spectrum
from arte.photometry.filters import Filters
from synphot.observation import Observation
from synphot import SpectralElement, Empirical1D
from morfeo.utils.constants import ELT

import warnings
warnings.filterwarnings('ignore')

In [6]:
#Compute flux at M1
filt_LO = Filters.ESO_ETC_H
zenith_angle = 30 * u.deg
airmass = 1 / np.cos(zenith_angle.to(u.rad))
f_vega = get_normalized_star_spectrum(spectral_type='vega', magnitude=0, filter_name=filt_LO)
# sky = EsoSkyCalc(airmass=airmass, moon_target_sep=45.0, moon_alt=45, observatory='3060m', wdelta=10)
sky = EsoSkyCalc(airmass=airmass, incl_moon='N', observatory='3060m')
sky_se = SpectralElement(Empirical1D, points=sky.lam, lookup_table=sky.trans)
lo_ch = MorfeoLowOrderChannelTransmissiveSystem_002()
filt = sky_se * lo_ch.as_transmissive_element().transmittance 
f_vega_at_LO = Observation(spec=f_vega, band=filt, force='taper', binset=f_vega.waveset)

area_subap = 1 * u.m**2
# area_subap = ELT.photometric_area / 4
counts = f_vega_at_LO.countrate(area=area_subap)
# print(f_vega_at_LO)
print(counts)
exp_time = 1 * u.s
# exp_time = 2 * u.ms
print((counts * exp_time).decompose() / 1e8) 

Note: skycalc_cli v.1.4 output wavelength unit is nm (it is μm in v.1.3)
1785666280.9253318 ct / s
17.85666280925332 ct


In [6]:
wv_H_min = 1.5 * u.um
wv_H_max = 1.8 * u.um

id_H_min = np.argwhere(wv==wv_H_min.to(u.Angstrom))[0][0]
id_H_max = np.argwhere(wv==wv_H_max.to(u.Angstrom))[0][0]
flux_mean_H = np.mean(flux_at_LO[id_H_min:id_H_max])
trans_lo_mean_H = np.mean(lo_trans[id_H_min:id_H_max])
print(wv[id_H_min].to(u.um), wv[id_H_max].to(u.um)) 
print('H band average transmission of LO channel: %s' %trans_lo_mean_H)
print('Flux at LO WFS: %s [e$^-$/m$^2$/s]' %flux_mean_H.value)

1.5 um 1.8 um
H band average transmission of LO channel: 0.30737934409877227
Flux at LO WFS: 305735.3379881788 [e$^-$/m$^2$/s]
